# Pokémon Battles — Ensemble XGBoost + LightGBM (Voting)
Notebook che implementa ensemble tra XGBoost e LightGBM con:
- **STESSE FEATURE** del notebook LightGBM ottimizzato
- **STESSI IPERPARAMETRI** dal notebook LightGBM
- 10-fold CV per entrambi i modelli
- Voting Classifier (hard voting + soft voting)
- Weighted voting con ottimizzazione pesi
- Submission finale

## Load Data

In [1]:
import json
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# --- Percorsi ---
train_file_path = 'train.jsonl'
test_file_path = 'test.jsonl'

def load_jsonl(path):
    data = []
    with open(path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

print('Caricamento dati...')
train_raw = load_jsonl(train_file_path)
test_raw = load_jsonl(test_file_path)
print(f'Train records: {len(train_raw)}, Test records: {len(test_raw)}')

Caricamento dati...
Train records: 10000, Test records: 5000
Train records: 10000, Test records: 5000


## Feature Engineering Functions
NOTA: Copia TUTTE le funzioni dal notebook LightGBM per garantire identiche feature

In [3]:
import math
from collections import Counter

# === TYPE CHART (Gen 1) ===
TYPE_CHART = {
    'normal': {'rock': 0.5, 'ghost': 0},
    'fire': {'fire': 0.5, 'water': 0.5, 'grass': 2, 'ice': 2, 'bug': 2, 'rock': 0.5, 'dragon': 0.5},
    'water': {'fire': 2, 'water': 0.5, 'grass': 0.5, 'ground': 2, 'rock': 2, 'dragon': 0.5},
    'grass': {'fire': 0.5, 'water': 2, 'grass': 0.5, 'poison': 0.5, 'ground': 2, 'flying': 0.5, 'bug': 0.5, 'rock': 2, 'dragon': 0.5},
    'electric': {'water': 2, 'grass': 0.5, 'electric': 0.5, 'ground': 0, 'flying': 2, 'dragon': 0.5},
    'ice': {'fire': 0.5, 'water': 0.5, 'grass': 2, 'ground': 2, 'flying': 2, 'dragon': 2},
    'fighting': {'normal': 2, 'ice': 2, 'poison': 0.5, 'flying': 0.5, 'psychic': 0.5, 'bug': 0.5, 'rock': 2, 'ghost': 0},
    'poison': {'grass': 2, 'poison': 0.5, 'ground': 0.5, 'bug': 2, 'rock': 0.5, 'ghost': 0.5},
    'ground': {'fire': 2, 'grass': 0.5, 'electric': 2, 'poison': 2, 'flying': 0, 'bug': 0.5, 'rock': 2},
    'flying': {'grass': 2, 'electric': 0.5, 'fighting': 2, 'bug': 2, 'rock': 0.5},
    'psychic': {'fighting': 2, 'poison': 2, 'psychic': 0.5, 'ghost': 0},
    'bug': {'fire': 0.5, 'grass': 2, 'fighting': 0.5, 'poison': 2, 'flying': 0.5, 'psychic': 2, 'ghost': 0.5},
    'rock': {'fire': 2, 'ice': 2, 'fighting': 0.5, 'ground': 0.5, 'flying': 2, 'bug': 2},
    'ghost': {'normal': 0, 'psychic': 0, 'ghost': 2},
    'dragon': {'dragon': 2}
}

def get_effectiveness(attack_type: str, defense_types: list) -> float:
    if not attack_type or not defense_types:
        return 1.0
    eff = 1.0
    for d in defense_types:
        eff *= TYPE_CHART.get(attack_type, {}).get(d, 1.0)
    return eff

def calculate_type_advantage(team1: list, team2_lead: dict) -> dict:
    out = {'p1_vs_lead_avg_effectiveness': 0.0, 'p1_vs_lead_max_effectiveness': 0.0, 'p1_super_effective_options': 0}
    if not team1 or not team2_lead:
        return out
    lead_types = [t.lower() for t in team2_lead.get('types', [])]
    if not lead_types:
        return out
    effs = []
    for p in team1:
        p_types = [t.lower() for t in p.get('types', [])]
        max_eff = 0.0
        for pt in p_types:
            max_eff = max(max_eff, get_effectiveness(pt, lead_types))
        effs.append(max_eff)
    if not effs:
        return out
    out['p1_vs_lead_avg_effectiveness'] = float(np.mean(effs))
    out['p1_vs_lead_max_effectiveness'] = float(np.max(effs))
    out['p1_super_effective_options'] = int(sum(1 for e in effs if e >= 2))
    return out

def _entropy(counter: Counter) -> float:
    total = sum(counter.values())
    if total == 0:
        return 0.0
    ent = 0.0
    for v in counter.values():
        p = v / total
        if p > 0:
            ent -= p * math.log(p, 2)
    return ent

def team_aggregate_features(team: list, prefix: str = 'p1_') -> dict:
    stats = ['base_hp','base_atk','base_def','base_spa','base_spd','base_spe']
    out = {}
    vals = {s: [] for s in stats}
    levels = []
    types_counter = Counter()
    names = []
    for p in team:
        names.append(p.get('name',''))
        for s in stats:
            vals[s].append(p.get(s, 0))
        levels.append(p.get('level', 0))
        for t in p.get('types', []):
            types_counter[t.lower()] += 1
    for s in stats:
        arr = np.array(vals[s], dtype=float)
        out[f'{prefix}{s}_sum'] = float(arr.sum())
        out[f'{prefix}{s}_mean'] = float(arr.mean())
        out[f'{prefix}{s}_max'] = float(arr.max())
        out[f'{prefix}{s}_min'] = float(arr.min())
        out[f'{prefix}{s}_std'] = float(arr.std())
    level_arr = np.array(levels, dtype=float)
    out[f'{prefix}level_mean'] = float(level_arr.mean()) if level_arr.size else 0.0
    out[f'{prefix}level_sum'] = float(level_arr.sum()) if level_arr.size else 0.0
    out[f'{prefix}n_unique_types'] = int(len(types_counter))
    common_types = ['normal','fire','water','electric','grass','psychic','ice','dragon','rock','ground','flying']
    for t in common_types:
        out[f'{prefix}type_{t}_count'] = int(types_counter.get(t, 0))
    out[f'{prefix}lead_name'] = names[0] if names else ''
    out[f'{prefix}n_unique_names'] = int(len(set(names)))
    out[f'{prefix}type_entropy'] = float(_entropy(types_counter))
    spe_arr = np.array(vals['base_spe'], dtype=float)
    out[f'{prefix}spe_p25'] = float(np.percentile(spe_arr, 25)) if spe_arr.size else 0.0
    out[f'{prefix}spe_p50'] = float(np.percentile(spe_arr, 50)) if spe_arr.size else 0.0
    out[f'{prefix}spe_p75'] = float(np.percentile(spe_arr, 75)) if spe_arr.size else 0.0
    return out

def lead_vs_lead_features(p1_lead: dict, p2_lead: dict) -> dict:
    out = {}
    stats = ['base_hp','base_atk','base_def','base_spa','base_spd','base_spe']
    for s in stats:
        out[f'lead_diff_{s}'] = float(p1_lead.get(s,0) - p2_lead.get(s,0))
    out['lead_speed_advantage'] = float(p1_lead.get('base_spe',0) - p2_lead.get('base_spe',0))
    p1_types = [t.lower() for t in p1_lead.get('types', [])]
    p2_types = [t.lower() for t in p2_lead.get('types', [])]
    max_eff = 0.0
    for pt in p1_types:
        max_eff = max(max_eff, get_effectiveness(pt, p2_types))
    out['lead_p1_vs_p2_effectiveness'] = float(max_eff)
    return out

def lead_aggregate_features(pokemon: dict, prefix: str = 'p2_lead_') -> dict:
    out = {}
    stats = ['base_hp','base_atk','base_def','base_spa','base_spd','base_spe']
    for s in stats:
        out[f'{prefix}{s}'] = float(pokemon.get(s,0))
    out[f'{prefix}level'] = int(pokemon.get('level',0))
    types = [x.lower() for x in pokemon.get('types', [])]
    common_types = ['normal','fire','water','electric','grass','psychic','ice','dragon','rock','ground','flying']
    for t in common_types:
        out[f'{prefix}type_{t}'] = int(t in types)
    out[f'{prefix}name'] = pokemon.get('name','')
    out[f'{prefix}n_unique_types'] = int(len(set(types)))
    return out

def quick_boost_features_v2(record: dict) -> dict:
    """Feature ad alto impatto senza bisogno team completo P2"""
    out = {}
    p1_team = record.get('p1_team_details', [])
    p2_lead = record.get('p2_lead_details', {})
    timeline = record.get('battle_timeline', [])
    
    if not p1_team:
        return out
    
    # 1. Speed Control
    p2_lead_spe = p2_lead.get('base_spe', 0)
    faster_count = sum(1 for p in p1_team if p.get('base_spe', 0) > p2_lead_spe)
    slower_count = sum(1 for p in p1_team if p.get('base_spe', 0) <= p2_lead_spe)
    
    out['p1_faster_than_lead_count'] = faster_count
    out['p1_slower_than_lead_count'] = slower_count
    out['p1_speed_control_ratio'] = faster_count / max(1, len(p1_team))
    
    # 2. Bulk vs Lead
    p1_avg_bulk = np.mean([
        p.get('base_hp', 0) * (p.get('base_def', 0) + p.get('base_spd', 0)) 
        for p in p1_team
    ])
    p2_lead_bulk = p2_lead.get('base_hp', 1) * (
        p2_lead.get('base_def', 1) + p2_lead.get('base_spd', 1)
    )
    out['p1_avg_bulk_vs_lead'] = p1_avg_bulk / max(p2_lead_bulk, 1)
    
    # 3. Offensive Power
    p1_total_atk = sum(p.get('base_atk', 0) + p.get('base_spa', 0) for p in p1_team)
    p2_lead_offense = p2_lead.get('base_atk', 0) + p2_lead.get('base_spa', 0)
    out['p1_total_offense'] = p1_total_atk
    out['p1_offense_advantage'] = p1_total_atk / max(p2_lead_offense, 1)
    
    # 4. Type Coverage Score
    p2_lead_types = [t.lower() for t in p2_lead.get('types', [])]
    if p2_lead_types:
        coverage_scores = []
        for p in p1_team:
            p_types = [t.lower() for t in p.get('types', [])]
            max_eff = max([get_effectiveness(pt, p2_lead_types) for pt in p_types] or [1.0])
            coverage_scores.append(max_eff)
        
        out['p1_avg_effectiveness_vs_lead'] = float(np.mean(coverage_scores))
        out['p1_max_effectiveness_vs_lead'] = float(np.max(coverage_scores))
        out['p1_se_count_vs_lead'] = sum(1 for s in coverage_scores if s >= 2.0)
        out['p1_weak_count_vs_lead'] = sum(1 for s in coverage_scores if s <= 0.5)
    
    # 5. First blood
    if timeline:
        first_p1_ko = False
        first_p2_ko = False
        
        for turn in timeline[:30]:
            p1_state = turn.get('p1_pokemon_state', {})
            p2_state = turn.get('p2_pokemon_state', {})
            
            if not first_p2_ko and p2_state.get('fainted'):
                first_p1_ko = True
                out['p1_first_blood'] = 1
                out['p1_first_blood_turn'] = turn.get('turn', 0)
                break
            
            if not first_p1_ko and p1_state.get('fainted'):
                first_p2_ko = True
                out['p1_first_blood'] = 0
                out['p1_first_blood_turn'] = turn.get('turn', 0)
                break
        
        if not first_p1_ko and not first_p2_ko:
            out['p1_first_blood'] = -1
            out['p1_first_blood_turn'] = 0
    
    # 6. Team Level Advantage
    p1_avg_level = np.mean([p.get('level', 50) for p in p1_team])
    p2_lead_level = p2_lead.get('level', 50)
    out['p1_avg_level_advantage'] = p1_avg_level - p2_lead_level
    
    # 7. Stat Product
    p1_stat_products = []
    for p in p1_team:
        stat_product = (
            p.get('base_hp', 1) * 
            p.get('base_atk', 1) * 
            p.get('base_def', 1) * 
            p.get('base_spa', 1) * 
            p.get('base_spd', 1) * 
            p.get('base_spe', 1)
        )
        p1_stat_products.append(stat_product)
    
    out['p1_avg_stat_product'] = float(np.mean(p1_stat_products))
    out['p1_max_stat_product'] = float(np.max(p1_stat_products))
    
    p2_lead_stat_product = (
        p2_lead.get('base_hp', 1) * 
        p2_lead.get('base_atk', 1) * 
        p2_lead.get('base_def', 1) * 
        p2_lead.get('base_spa', 1) * 
        p2_lead.get('base_spd', 1) * 
        p2_lead.get('base_spe', 1)
    )
    out['p1_stat_product_advantage'] = out['p1_avg_stat_product'] / max(p2_lead_stat_product, 1)
    
    return out

def summary_from_timeline(timeline: list, p1_team: list) -> dict:
    out = {}
    if not timeline:
        return {'tl_p1_moves':0,'tl_p2_moves':0,'tl_p1_est_damage':0.0,'tl_p2_est_damage':0.0,'damage_diff':0.0}
    
    p1_moves = p2_moves = 0
    p1_damage = p2_damage = 0.0
    p1_last_active = p2_last_active = ''
    p1_last_hp = p2_last_hp = np.nan
    p1_fainted = p2_fainted = 0
    p1_fainted_names = set()
    p2_fainted_names = set()
    last_p1_hp = {}
    last_p2_hp = {}
    
    for turn in timeline[:30]:
        p1_state = turn.get('p1_pokemon_state',{}) or {}
        p2_state = turn.get('p2_pokemon_state',{}) or {}
        
        if p1_state.get('name'):
            p1_last_active = p1_state.get('name')
        if p2_state.get('name'):
            p2_last_active = p2_state.get('name')
        
        if p1_state.get('fainted') and p1_state.get('name') not in p1_fainted_names:
            p1_fainted += 1
            p1_fainted_names.add(p1_state.get('name'))
        if p2_state.get('fainted') and p2_state.get('name') not in p2_fainted_names:
            p2_fainted += 1
            p2_fainted_names.add(p2_state.get('name'))
        
        p2_name, p2_hp = p2_state.get('name'), p2_state.get('hp_pct')
        if p2_name and p2_hp is not None:
            prev_hp = last_p2_hp.get(p2_name)
            if prev_hp is not None:
                delta = max(0.0, prev_hp - p2_hp)
                p1_damage += delta
            last_p2_hp[p2_name] = p2_hp
        
        p1_name, p1_hp = p1_state.get('name'), p1_state.get('hp_pct')
        if p1_name and p1_hp is not None:
            prev_hp = last_p1_hp.get(p1_name)
            if prev_hp is not None:
                delta = max(0.0, prev_hp - p1_hp)
                p2_damage += delta
            last_p1_hp[p1_name] = p1_hp
        
        if turn.get('p1_move_details'):
            p1_moves += 1
        if turn.get('p2_move_details'):
            p2_moves += 1
        
        p1_last_hp = p1_state.get('hp_pct', np.nan)
        p2_last_hp = p2_state.get('hp_pct', np.nan)
    
    out['tl_p1_moves'] = int(p1_moves)
    out['tl_p2_moves'] = int(p2_moves)
    out['tl_p1_est_damage'] = float(p1_damage)
    out['tl_p2_est_damage'] = float(p2_damage)
    out['tl_p1_fainted'] = int(p1_fainted)
    out['tl_p2_fainted'] = int(p2_fainted)
    out['damage_diff'] = float(p1_damage - p2_damage)
    out['fainted_diff'] = int(p1_fainted - p2_fainted)
    out['tl_p1_last_hp'] = float(p1_last_hp) if not np.isnan(p1_last_hp) else 0.0
    out['tl_p2_last_hp'] = float(p2_last_hp) if not np.isnan(p2_last_hp) else 0.0
    out['tl_p1_last_active'] = p1_last_active
    out['tl_p2_last_active'] = p2_last_active
    
    return out

def ability_features(team: list, prefix: str) -> dict:
    immunity_abilities = {'levitate':0,'volt_absorb':0,'water_absorb':0,'flash_fire':0}
    stat_drop_abilities = {'intimidate':0}
    weather_abilities = {'drought':0,'drizzle':0,'sand_stream':0}
    out = {}
    for pokemon in team:
        ability = (pokemon.get('ability','') or '').lower().replace(' ','_')
        if ability in immunity_abilities:
            immunity_abilities[ability] += 1
        if ability in stat_drop_abilities:
            stat_drop_abilities[ability] += 1
        if ability in weather_abilities:
            weather_abilities[ability] += 1
    for ability,count in immunity_abilities.items():
        out[f'{prefix}ability_{ability}_count'] = int(count)
    for ability,count in stat_drop_abilities.items():
        out[f'{prefix}ability_{ability}_count'] = int(count)
    for ability,count in weather_abilities.items():
        out[f'{prefix}ability_{ability}_count'] = int(count)
    out[f'{prefix}total_immunity_abilities'] = int(sum(immunity_abilities.values()))
    out[f'{prefix}total_stat_drop_abilities'] = int(sum(stat_drop_abilities.values()))
    return out

def prepare_record_features(record: dict, max_turns: int = 30) -> dict:
    out = {}
    out['battle_id'] = record.get('battle_id')
    if 'player_won' in record:
        out['player_won'] = int(bool(record.get('player_won')))
    
    p1_team = record.get('p1_team_details', [])
    out.update(team_aggregate_features(p1_team, prefix='p1_'))
    
    p2_lead = record.get('p2_lead_details', {})
    out.update(lead_aggregate_features(p2_lead, prefix='p2_lead_'))
    out.update(ability_features(p1_team, prefix='p1_'))
    
    p1_lead = p1_team[0] if p1_team else {}
    out.update(lead_vs_lead_features(p1_lead, p2_lead))
    out.update(ability_features([p2_lead], prefix='p2_lead_'))
    
    tl = record.get('battle_timeline', [])
    out.update(summary_from_timeline(tl[:max_turns], p1_team))
    out.update(quick_boost_features_v2(record))
    out.update(calculate_type_advantage(p1_team, p2_lead))
    
    # Derived features
    out['team_hp_sum_minus_p2lead_hp'] = out.get('p1_base_hp_sum', 0) - out.get('p2_lead_base_hp', 0)
    out['speed_advantage'] = out.get('p1_base_spe_sum', 0) - out.get('p2_lead_base_spe', 0)
    
    p2_team = record.get('p2_team_details', [])
    if p2_team:
        out.update(team_aggregate_features(p2_team, prefix='p2_'))
        out['team_hp_sum_diff'] = out.get('p1_base_hp_sum',0) - out.get('p2_base_hp_sum',0)
        out['team_spe_mean_diff'] = out.get('p1_base_spe_mean',0) - out.get('p2_base_spe_mean',0)
    
    return out

def create_features_from_raw(data: list) -> pd.DataFrame:
    rows = []
    for b in tqdm(data, desc='Feature Engineering'):
        try:
            feat = prepare_record_features(b, max_turns=30)
            if 'battle_id' not in feat:
                feat['battle_id'] = b.get('battle_id')
            rows.append(feat)
        except Exception as e:
            rows.append({'battle_id': b.get('battle_id'), 'error': 1})
    df = pd.DataFrame(rows)
    if 'player_won' in df.columns:
        df['player_won'] = df['player_won'].astype(int)
    return df.fillna(0)

print("✅ Feature engineering functions loaded")

✅ Feature engineering functions loaded


## Create Features

In [4]:
train_df = create_features_from_raw(train_raw)
test_df = create_features_from_raw(test_raw)
print('Feature shape train/test:', train_df.shape, test_df.shape)
display(train_df.head())

Feature Engineering: 100%|██████████| 10000/10000 [00:06<00:00, 1467.20it/s]

Feature Engineering: 100%|██████████| 5000/5000 [00:03<00:00, 1584.34it/s]



Feature shape train/test: (10000, 133) (5000, 132)


,battle_id,player_won,p1_base_hp_sum,p1_base_hp_mean,p1_base_hp_max,p1_base_hp_min,p1_base_hp_std,p1_base_atk_sum,p1_base_atk_mean,p1_base_atk_max,...,p1_first_blood_turn,p1_avg_level_advantage,p1_avg_stat_product,p1_max_stat_product,p1_stat_product_advantage,p1_vs_lead_avg_effectiveness,p1_vs_lead_max_effectiveness,p1_super_effective_options,team_hp_sum_minus_p2lead_hp,speed_advantage
0,0,1,695.0,115.833333,250.0,55.0,69.367179,435.0,72.500000,110.0,...,0,0.0,3.170415e+11,6.592480e+11,0.720754,1.083333,2.0,1,635.0,365.0
1,1,1,740.0,123.333333,250.0,65.0,64.204534,435.0,72.500000,110.0,...,0,0.0,2.399565e+11,6.592480e+11,0.886622,1.000000,1.0,0,685.0,250.0
2,2,1,745.0,124.166667,250.0,60.0,64.382753,505.0,84.166667,130.0,...,0,0.0,2.932361e+11,6.592480e+11,85.111624,1.000000,1.0,0,495.0,345.0
3,3,1,730.0,121.666667,250.0,60.0,65.362239,465.0,77.500000,110.0,...,0,0.0,4.504200e+11,1.075781e+12,1.172854,1.000000,1.0,0,655.0,345.0
4,4,1,685.0,114.166667,250.0,50.0,70.794107,455.0,75.833333,110.0,...,0,0.0,3.157984e+11,6.592480e+11,0.717928,1.083333,2.0,1,625.0,320.0


## Preprocessing - STESSE FEATURE del notebook LightGBM

In [ ]:
# ✅ USA LE STESSE FEATURE del notebook LightGBM ottimizzato
# Copia esatta dal notebook pokemon_lightgbm_cv.ipynb

exclude_cols = ['battle_id', 'player_won']
string_cols = train_df.select_dtypes(include=['object']).columns.tolist()
exclude_cols.extend(string_cols)

ALL_NUMERIC_FEATURES = [c for c in train_df.columns if c not in exclude_cols]

# ⚠️ IMPORTANTE: Usa FEATURES_FINAL se hai fatto feature selection nel notebook LightGBM
# Altrimenti usa ALL_NUMERIC_FEATURES
# Per ora uso tutte le feature (come nel notebook LightGBM originale)
FEATURES = ALL_NUMERIC_FEATURES

print(f'✅ Usando {len(FEATURES)} feature (identiche al notebook LightGBM)')

# Preprocessing identico
num_df = train_df[FEATURES].astype(float).replace([np.inf, -np.inf], np.nan)
medians = num_df.median()
train_imputed = num_df.fillna(medians)
train_preproc_df = train_imputed.copy()

y = train_df['player_won'].astype(int).values
X = train_preproc_df.values

print('Preprocessing completato.')
print('Dataset size:', X.shape)

# Allinea test
test_aligned = test_df.reindex(columns=FEATURES, fill_value=np.nan).astype(float).replace([np.inf, -np.inf], np.nan)
test_imputed = test_aligned.fillna(medians)
test_preproc_df = pd.DataFrame(test_imputed.values, columns=FEATURES, index=test_df.index)

Num FEATURES numeriche: 127
Preprocessing completato.
Dataset completo size: 10000
Features: 127


## Model Hyperparameters - IDENTICI al notebook LightGBM

In [ ]:
# ✅ IPERPARAMETRI IDENTICI dal notebook pokemon_lightgbm_cv.ipynb
# Questi sono i best params trovati con Optuna nel notebook LightGBM

lgb_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'seed': 42,
    'learning_rate': 0.048286,
    'n_estimators': 600,
    'num_leaves': 74,
    'max_depth': 7,
    'min_child_samples': 95,
    'min_child_weight': 0.046044,
    'reg_alpha': 0.001554,
    'reg_lambda': 0.000040,
    'subsample': 0.578947,
    'colsample_bytree': 0.575072,
    'subsample_freq': 2,
    'min_split_gain': 0.741598,
    'max_bin': 320
}

# ✅ XGBoost params - equivalenti configurati per matching
# Mappo i parametri LightGBM a XGBoost per coerenza
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'seed': 42,
    'learning_rate': 0.048286,  # STESSO di LightGBM
    'n_estimators': 600,         # STESSO di LightGBM
    'max_depth': 7,              # STESSO di LightGBM
    'min_child_weight': 0.046,   # Equivalente a min_child_weight LightGBM
    'gamma': 0.741,              # Equivalente a min_split_gain
    'subsample': 0.579,          # STESSO di LightGBM
    'colsample_bytree': 0.575,   # STESSO di LightGBM
    'reg_alpha': 0.001554,       # STESSO di LightGBM
    'reg_lambda': 0.000040,      # STESSO di LightGBM
    'tree_method': 'hist',
    'max_bin': 320,              # STESSO di LightGBM
    'random_state': 42
}

print("✅ Hyperparameters configurati (IDENTICI al notebook LightGBM)")
print(f"\nLightGBM params: {lgb_params}")
print(f"\nXGBoost params: {xgb_params}")

## 10-Fold CV - LightGBM

In [6]:
print("="*70)
print("10-FOLD CV - LIGHTGBM")
print("="*70)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

lgb_accuracies = []
lgb_oof_preds = np.zeros(len(X))  # Out-of-fold predictions

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)]
    )
    
    y_pred_val = clf.predict(X_val)
    val_acc = accuracy_score(y_val, y_pred_val)
    lgb_accuracies.append(val_acc)
    
    # Salva OOF predictions per ensemble
    lgb_oof_preds[val_idx] = clf.predict_proba(X_val)[:, 1]
    
    print(f'LightGBM Fold {fold_idx}: val_acc={val_acc*100:.2f}%')

lgb_mean_acc = np.mean(lgb_accuracies)
print(f'\n✅ LightGBM Mean CV Accuracy: {lgb_mean_acc*100:.2f}% ± {np.std(lgb_accuracies)*100:.2f}%')

10-FOLD CV - LIGHTGBM


NameError: name 'lgb_params' is not defined

## 10-Fold CV - XGBoost

In [ ]:
print("="*70)
print("10-FOLD CV - XGBOOST")
print("="*70)

xgb_accuracies = []
xgb_oof_preds = np.zeros(len(X))

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    clf = xgb.XGBClassifier(**xgb_params)
    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=30,
        verbose=False
    )
    
    y_pred_val = clf.predict(X_val)
    val_acc = accuracy_score(y_val, y_pred_val)
    xgb_accuracies.append(val_acc)
    
    # Salva OOF predictions
    xgb_oof_preds[val_idx] = clf.predict_proba(X_val)[:, 1]
    
    print(f'XGBoost Fold {fold_idx}: val_acc={val_acc*100:.2f}%')

xgb_mean_acc = np.mean(xgb_accuracies)
print(f'\n✅ XGBoost Mean CV Accuracy: {xgb_mean_acc*100:.2f}% ± {np.std(xgb_accuracies)*100:.2f}%')

## Ensemble - Hard Voting

In [ ]:
print("="*70)
print("ENSEMBLE - HARD VOTING")
print("="*70)

# Hard voting: predizione basata su maggioranza
lgb_hard = (lgb_oof_preds >= 0.5).astype(int)
xgb_hard = (xgb_oof_preds >= 0.5).astype(int)

# Majority vote
hard_vote_preds = ((lgb_hard + xgb_hard) >= 1).astype(int)
hard_vote_acc = accuracy_score(y, hard_vote_preds)

print(f"Hard Voting Accuracy: {hard_vote_acc*100:.2f}%")
print(f"LightGBM solo: {lgb_mean_acc*100:.2f}%")
print(f"XGBoost solo:  {xgb_mean_acc*100:.2f}%")
print(f"Improvement: {(hard_vote_acc - max(lgb_mean_acc, xgb_mean_acc))*100:.2f}%")

## Ensemble - Soft Voting (Simple Average)

In [ ]:
print("="*70)
print("ENSEMBLE - SOFT VOTING (SIMPLE AVERAGE)")
print("="*70)

# Soft voting: media delle probabilità
soft_vote_probs = (lgb_oof_preds + xgb_oof_preds) / 2
soft_vote_preds = (soft_vote_probs >= 0.5).astype(int)
soft_vote_acc = accuracy_score(y, soft_vote_preds)

print(f"Soft Voting (0.5/0.5) Accuracy: {soft_vote_acc*100:.2f}%")
print(f"Improvement over best single: {(soft_vote_acc - max(lgb_mean_acc, xgb_mean_acc))*100:.2f}%")

## Ensemble - Weighted Soft Voting (Optimization)

In [ ]:
print("="*70)
print("ENSEMBLE - WEIGHTED SOFT VOTING (GRID SEARCH)")
print("="*70)

# Trova pesi ottimali con grid search
best_weight = 0.5
best_weighted_acc = 0.0

weights_to_try = np.arange(0.0, 1.01, 0.05)
results = []

for w_lgb in weights_to_try:
    w_xgb = 1.0 - w_lgb
    weighted_probs = w_lgb * lgb_oof_preds + w_xgb * xgb_oof_preds
    weighted_preds = (weighted_probs >= 0.5).astype(int)
    acc = accuracy_score(y, weighted_preds)
    results.append({'w_lgb': w_lgb, 'w_xgb': w_xgb, 'accuracy': acc})
    
    if acc > best_weighted_acc:
        best_weighted_acc = acc
        best_weight = w_lgb

results_df = pd.DataFrame(results)
print(f"\n✅ Best Weights Found:")
print(f"   LightGBM weight: {best_weight:.2f}")
print(f"   XGBoost weight:  {1-best_weight:.2f}")
print(f"   Accuracy: {best_weighted_acc*100:.2f}%")

print(f"\nTop 10 weight combinations:")
display(results_df.sort_values('accuracy', ascending=False).head(10))

# Salva risultati
results_df.to_csv('ensemble_weight_search.csv', index=False)
print(f"\n✅ Weight search results saved to 'ensemble_weight_search.csv'")

In [ ]:
# Plot weight vs accuracy
try:
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['w_lgb'], results_df['accuracy'] * 100, marker='o', linewidth=2)
    plt.axvline(best_weight, color='red', linestyle='--', label=f'Best weight={best_weight:.2f}')
    plt.axhline(lgb_mean_acc * 100, color='blue', linestyle='--', alpha=0.5, label=f'LGB only={lgb_mean_acc*100:.2f}%')
    plt.axhline(xgb_mean_acc * 100, color='green', linestyle='--', alpha=0.5, label=f'XGB only={xgb_mean_acc*100:.2f}%')
    plt.xlabel('LightGBM Weight')
    plt.ylabel('Accuracy (%)')
    plt.title('Ensemble Accuracy vs Weight (LightGBM)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('ensemble_weight_curve.png', dpi=150)
    print("✅ Plot saved to 'ensemble_weight_curve.png'")
except Exception as e:
    print(f"⚠️ Plot failed: {e}")

## Final 10-Fold CV - Voting Ensemble

In [ ]:
print("="*70)
print("FINAL 10-FOLD CV - VOTING ENSEMBLE")
print("="*70)

ensemble_accuracies = []
ensemble_oof_preds = np.zeros(len(X))

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    # Train both models
    lgb_clf = lgb.LGBMClassifier(**lgb_params)
    xgb_clf = xgb.XGBClassifier(**xgb_params)
    
    lgb_clf.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], 
                callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)])
    xgb_clf.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], 
                early_stopping_rounds=30, verbose=False)
    
    # Weighted soft voting con best weights
    lgb_probs = lgb_clf.predict_proba(X_val)[:, 1]
    xgb_probs = xgb_clf.predict_proba(X_val)[:, 1]
    
    ensemble_probs = best_weight * lgb_probs + (1 - best_weight) * xgb_probs
    ensemble_preds = (ensemble_probs >= 0.5).astype(int)
    
    val_acc = accuracy_score(y_val, ensemble_preds)
    ensemble_accuracies.append(val_acc)
    ensemble_oof_preds[val_idx] = ensemble_probs
    
    print(f'Ensemble Fold {fold_idx}: val_acc={val_acc*100:.2f}%')

ensemble_mean_acc = np.mean(ensemble_accuracies)
print(f'\n' + '='*70)
print(f'FINAL RESULTS (usando STESSI params del notebook LightGBM):')
print(f'='*70)
print(f'LightGBM:     {lgb_mean_acc*100:.2f}% ± {np.std(lgb_accuracies)*100:.2f}%')
print(f'XGBoost:      {xgb_mean_acc*100:.2f}% ± {np.std(xgb_accuracies)*100:.2f}%')
print(f'Ensemble:     {ensemble_mean_acc*100:.2f}% ± {np.std(ensemble_accuracies)*100:.2f}%')
print(f'\nImprovement:  +{(ensemble_mean_acc - max(lgb_mean_acc, xgb_mean_acc))*100:.2f}%')
print(f'\n📊 Config: {len(FEATURES)} features, LGB weight={best_weight:.2f}')

## Make Submission

In [ ]:
print("="*70)
print("SUBMISSION - ENSEMBLE VOTING")
print("="*70)

# Train final models su tutto il dataset
print("Training LightGBM on full dataset...")
final_lgb = lgb.LGBMClassifier(**lgb_params)
final_lgb.fit(X, y)

print("Training XGBoost on full dataset...")
final_xgb = xgb.XGBClassifier(**xgb_params)
final_xgb.fit(X, y)

# Predict su test con weighted voting
print("Predicting on test set...")
X_test_matrix = test_preproc_df.values

lgb_test_probs = final_lgb.predict_proba(X_test_matrix)[:, 1]
xgb_test_probs = final_xgb.predict_proba(X_test_matrix)[:, 1]

# Ensemble con best weights
ensemble_test_probs = best_weight * lgb_test_probs + (1 - best_weight) * xgb_test_probs
ensemble_test_preds = (ensemble_test_probs >= 0.5).astype(int)

# Crea submission
submission_df = pd.DataFrame({
    'battle_id': test_df['battle_id'].astype(np.int64),
    'player_won': ensemble_test_preds.astype(np.int64)
})

submission_path = 'submission_ensemble_voting.csv'
submission_df.to_csv(submission_path, index=False)

print(f"\n✅ Submission saved to '{submission_path}'")
print(f"Models: LightGBM ({best_weight:.2f}) + XGBoost ({1-best_weight:.2f})")
print(f"Features: {len(FEATURES)} (identiche al notebook LightGBM)")
print(f"Expected CV Accuracy: {ensemble_mean_acc*100:.2f}% ± {np.std(ensemble_accuracies)*100:.2f}%")
print(f"\nPrediction distribution:")
print(submission_df['player_won'].value_counts())
print(f"\nPreview:")
display(submission_df.head(10))

In [ ]:
# Bonus: Salva anche predizioni individuali per confronto
lgb_submission = pd.DataFrame({
    'battle_id': test_df['battle_id'].astype(np.int64),
    'player_won': final_lgb.predict(X_test_matrix).astype(np.int64)
})
lgb_submission.to_csv('submission_lightgbm_only.csv', index=False)

xgb_submission = pd.DataFrame({
    'battle_id': test_df['battle_id'].astype(np.int64),
    'player_won': final_xgb.predict(X_test_matrix).astype(np.int64)
})
xgb_submission.to_csv('submission_xgboost_only.csv', index=False)

print("✅ Individual model submissions also saved:")
print("   - submission_lightgbm_only.csv")
print("   - submission_xgboost_only.csv")
print("   - submission_ensemble_voting.csv (RECOMMENDED)")
print("\n📋 Summary:")
print(f"   • {len(FEATURES)} features (same as LightGBM notebook)")
print(f"   • LGB params: Optuna-optimized from LightGBM notebook")
print(f"   • XGB params: Mapped from LGB params for consistency")
print(f"   • Ensemble weight: {best_weight:.2f} LGB + {1-best_weight:.2f} XGB")